In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import time
import json

from torch.ao.quantization.backend_config.backend_config import NAME_DICT_KEY


In [6]:
def step3_push_excel(Name, Dosage, Component, Way_Taking, Type_Form, Zheng_Form, Illness, row_index):
    """
    将爬取到的内容放入表格
    :param row_index:
    :param Name:
    :param Dosage:
    :param Component:
    :param Way_Taking:
    :param Type_Form:
    :param Zheng_Form:
    :param Illness:
    :return:
    """
    data_ETCM = pd.read_excel("D:/Snoopy-Auxiliary-Code/result/result_fj.xlsx", sheet_name=0)

    data_ETCM.loc[row_index, '方剂名称'] = Name[0]
    data_ETCM.loc[row_index, '剂型'] = Dosage[0]
    data_ETCM.loc[row_index, '使用方法'] = Way_Taking[0]
    data_ETCM.loc[row_index, '方剂类型'] = Type_Form[0]
    data_ETCM.loc[row_index, '对证'] = Zheng_Form[0]
    data_ETCM.loc[row_index, '临床表现'] = Illness[0]

    for i in range(len(Component)):
        data_ETCM.loc[row_index, i+6] = Component[i]

    data_ETCM.to_excel("D:/Snoopy-Auxiliary-Code/result/result_fj.xlsx", index=False)

In [15]:
def step4(Name, protein, row_index):

    global data_ETCM

    if protein != "":
        Name = [Name[0]] * len(protein)

        data = {
        "source": Name,
        "target": protein
    }
        df = pd.DataFrame(data)
        df = df[df['target'].isin(targets)]
        data_ETCM = pd.concat([data_ETCM, df], ignore_index=True)

    else:
        return 0

In [16]:
def step2_get_infor(page_content, TCM_index):
    """
    获取各类方剂及其对应信息
    :param TCM_index:
    :param page_content: 网页源码
    :return:
    """
    # 获取方剂名称信息
    Name_Form_pattern = r"<a.*?href=.*?target='_blank'>(.*?)<\/a>"
    Name_Form_matches = re.findall(Name_Form_pattern, page_content)

    # 获取方剂剂型
    Dosage_Form_pattern = r'Dosage Form.*?>.*?>(.*?)<\/div>'
    Dosage_Form_matches = re.findall(Dosage_Form_pattern, page_content)

    # 获取方剂组成
    Component_Form_pattern = r"html\?id=.*?>(.*?)</a>"
    Component_Form_matches = re.findall(Component_Form_pattern, page_content)
    Component_Form_matches = Component_Form_matches[0:int(len(Component_Form_matches)/2)]

    # 获取服用方法
    Way_Taking_Form_pattern = r"Administration</div>.*?>(.*?)</div>"
    Way_Taking_Form_matches = re.findall(Way_Taking_Form_pattern, page_content)

    # 获取药物类型
    Type_Form_pattern = r"id='L778'.*?Type.*?>.*?>(.*?)</div>"
    Type_Form_matches = re.findall(Type_Form_pattern, page_content)

    # 获取药物证相
    Zheng_Form_pattern = r"id='L779'.*?Syndromes in Chinese.*?>.*?>(.*?)</div>"
    Zheng_Form_matches = re.findall(Zheng_Form_pattern, page_content)

    # 获取药物对症
    Illness_Form_pattern = r"id='L7711'.*?Indications in Chinese.*?>.*?>(.*?)</div>"
    Illness_Form_matches = re.findall(Illness_Form_pattern, page_content)

    protein_Form_pattern = r"href=.*?gene=.*?>(.*?)</a>"
    protein_Form_matches = re.findall(protein_Form_pattern, page_content)

    step4(Name_Form_matches, protein_Form_matches, TCM_index)

    #step3_push_excel(Name_Form_matches, Dosage_Form_matches, Component_Form_matches, Way_Taking_Form_matches, Type_Form_matches, Zheng_Form_matches,Illness_Form_matches, TCM_index)

    return 0

    #return Name_Form_matches, Dosage_Form_matches, Component_Form_matches, Way_Taking_Form_matches, Type_Form_matches, Zheng_Form_matches,Illness_Form_matches

with open("D:/Snoopy-Auxiliary-Code/result/result.txt", "r", encoding='utf-8')as f:
    content = f.read()
step2_get_infor(content, 4)

NameError: name 'data_ETCM' is not defined

In [18]:
def step1_visit_website(target_url, TCM_index):
    """
    访问网站链接并获取网页源码
    :param TCM_index:
    :param target_url:网址链接
    :return: 网页源代码的txt格式
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'}

    proxies = {"http": None, "https": None}
    res = requests.get(url, headers=headers,proxies=proxies)
    res.encoding = 'utf-8'

    soup = BeautifulSoup(res.content, 'html.parser')
    script_tags = soup.find_all('script')[10]

    script_tags = str(script_tags)[:30000]
    #print(script_tags)

    step2_get_infor(script_tags, TCM_index)
    return 0

    #return Name_Form_matches, Dosage_Form_matches, Component_Form_matches, Way_Taking_Form_matches, Type_Form_matches, Zheng_Form_matches, Illness_Form_matches

url = 'http://www.tcmip.cn/ETCM/index.php/Home/Index/fj_details.html?id='
step1_visit_website(url, 7)

0

# main函数

In [19]:
start_time = time.time()

base_url = 'http://www.tcmip.cn/ETCM/index.php/Home/Index/fj_details.html?id='

global data_ETCM
data_ETCM = pd.read_excel("D:/Snoopy-Auxiliary-Code/result/result_etcm.xlsx", sheet_name=0)

for i in range(2696, 3971):
    extra_url = str(i + 1)
    url = base_url + extra_url
    try:
        print("第", i+1, "个网站已成功爬取")
        url_content = step1_visit_website(url, i+1)
    except:
        print("第", i+1, "个网站爬取失败")
        url_content = step1_visit_website(url, i+1)

    if i % 300 == 0:
        data_ETCM.to_excel("D:/Snoopy-Auxiliary-Code/result/result_etcm.xlsx", index=False)

data_ETCM.to_excel("D:/Snoopy-Auxiliary-Code/result/result_etcm.xlsx", index=False)

end_time = time.time()
print(str(end_time - start_time))

第 2697 个网站已成功爬取
第 2698 个网站已成功爬取
第 2699 个网站已成功爬取
第 2700 个网站已成功爬取
第 2701 个网站已成功爬取
第 2702 个网站已成功爬取
第 2703 个网站已成功爬取
第 2704 个网站已成功爬取
第 2705 个网站已成功爬取
第 2706 个网站已成功爬取
第 2707 个网站已成功爬取
第 2708 个网站已成功爬取
第 2709 个网站已成功爬取
第 2710 个网站已成功爬取
第 2711 个网站已成功爬取
第 2712 个网站已成功爬取
第 2713 个网站已成功爬取
第 2714 个网站已成功爬取
第 2715 个网站已成功爬取
第 2716 个网站已成功爬取
第 2717 个网站已成功爬取
第 2718 个网站已成功爬取
第 2719 个网站已成功爬取
第 2720 个网站已成功爬取
第 2721 个网站已成功爬取
第 2722 个网站已成功爬取
第 2723 个网站已成功爬取
第 2724 个网站已成功爬取
第 2725 个网站已成功爬取
第 2726 个网站已成功爬取
第 2727 个网站已成功爬取
第 2728 个网站已成功爬取
第 2729 个网站已成功爬取
第 2730 个网站已成功爬取
第 2731 个网站已成功爬取
第 2732 个网站已成功爬取
第 2733 个网站已成功爬取
第 2734 个网站已成功爬取
第 2735 个网站已成功爬取
第 2736 个网站已成功爬取
第 2737 个网站已成功爬取
第 2738 个网站已成功爬取
第 2739 个网站已成功爬取
第 2740 个网站已成功爬取
第 2741 个网站已成功爬取
第 2742 个网站已成功爬取
第 2743 个网站已成功爬取
第 2744 个网站已成功爬取
第 2745 个网站已成功爬取
第 2746 个网站已成功爬取
第 2747 个网站已成功爬取
第 2748 个网站已成功爬取
第 2749 个网站已成功爬取
第 2750 个网站已成功爬取
第 2751 个网站已成功爬取
第 2752 个网站已成功爬取
第 2753 个网站已成功爬取
第 2754 个网站已成功爬取
第 2755 个网站已成功爬取
第 2756 个网站已成功爬取
第 2757 个网站已成功爬取
第 2758 个网站已成功爬取
第 2759 个

In [2]:
with open('D:\Snoopy\data\PPI\PPI.json', 'r') as file:
    json_data = json.load(file)

In [3]:
def extract_targets(json_data):
    all_targets = set()  # 使用集合去重
    for key, value in json_data.items():
        if isinstance(value, list):  # 确保值是列表
            all_targets.update(value)  # 将列表中的靶点添加到集合中
    return all_targets

In [4]:
targets = extract_targets(json_data)
print("所有出现过的靶点：", targets)

所有出现过的靶点： {'ACD10', 'RTN4R', 'CPSF7', 'TRPC1', 'DGCR8', 'ENOA', 'BSCL2', 'MAL', 'SP3', 'ITB8', 'CC115', 'ASF1A', 'GBRAP', 'RS27', 'TSHB', 'NDUA9', 'EXOS8', 'ACTY', 'PSA4', 'AAMP', 'DPOG2', 'LSM7', 'PDYN', 'UIMC1', 'PPLA', 'UPK3A', 'NCKPL', 'MYH9', 'CD151', 'RSPO2', 'SHB', 'CASP4', 'YLAT2', 'STON2', 'PGRP4', 'KISS1', 'CTTB2', 'TMM43', 'EVL', 'TERF1', 'VTNC', 'SLD5', 'KCC2B', 'LBR', 'TNR21', 'DYR1A', 'ADIPO', 'SDCB1', 'GDIR2', 'NFE2', 'LAT2', 'ARK73', 'ZIC1', 'TLN1', 'ATG7', 'TNNT3', 'TAB2', 'TBB1', 'RGS9', 'S29A3', 'PEX2', 'COT2', 'GLIS2', 'KLOTB', 'KI2L3', 'RD21L', 'RXRA', 'TMCO1', 'DUS2L', 'CHM4A', 'TSR1', 'SIR2', 'ZN609', 'SCFD2', 'PTGIS', 'MECP2', 'SHC1', 'UBP28', 'IL18R', 'SALL4', 'CSTP1', 'SYPH', 'ATPO', 'NOD1', 'RT31', 'NUCL', 'MX2', 'RTCB', 'SYMM', 'CO4B', 'SCNNG', 'FOXO4', 'S26A3', 'CXCL9', 'PADI4', 'BI1', 'RPP25', 'SP130', 'AAK1', 'SMAD3', 'PMEL', 'IAPP', 'MESD', 'CC74B', 'CIP4', 'MON2', 'DCNL4', 'ADT2', 'ADH1G', 'IF5AL', 'ASB6', 'GPR55', 'MEPE', 'RIMB1', 'BDP1', 'NXT1', 'YBOX

In [5]:
len(targets)

9829

In [20]:
data = pd.read_excel("D:/Snoopy-Auxiliary-Code/result/result_etcm.xlsx")
data.head()

,formula,target
0,阿归养血颗粒,UBC
1,阿归养血颗粒,TOP2A
2,阿归养血颗粒,RBP1
3,阿归养血颗粒,COX6C
4,阿归养血颗粒,INS


In [13]:
data_new = data[data['target'].isin(targets)]
data_new

,source,target
0,阿归养血颗粒,UBC
1,阿归养血颗粒,TOP2A
5,阿归养血颗粒,RBP1
12,阿归养血颗粒,COX6C
16,阿归养血颗粒,INS
...,...,...
329066,水陆二味片,SYK
329068,水陆二味片,SHBG
329071,水陆二味片,ALDH2
329079,水陆二味片,AHR


In [14]:
data_new.to_excel("D:/Snoopy-Auxiliary-Code/result/result_etcm.xlsx", index=False)

In [21]:
len(data)

134132

In [22]:
# 导出为 JSON 文件，每行作为一个独立的 JSON 对象
data.to_json('D:/Snoopy-Auxiliary-Code/result/output.json', orient='records', lines=False, force_ascii=False)